## LAB 2

download data, upload table

embed data
embed query
rag respond with context

In [2]:
! pip install --upgrad google-cloud-bigquery


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

ambiguous option: --upgrad (--upgrade, --upgrade-strategy?)


open command line, copy over command.
```
gsutil cp gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv ~/
```
(It get's buggy here)
You may also remove ! marks and run the next cell there as well if you wish.

In [1]:
# Upload data

!bq mk aurora_data
# !bq ls # verify dataset
!bq load \
  --autodetect \
  --source_format=CSV \
  --skip_leading_rows=1 \
  aurora_data.faqs_table \
  ./aurora-bay-faqs.csv

 # !bq ls aurora_data # verify tables

BigQuery error in mk operation: Dataset 'qwiklabs-
gcp-01-14276dd314fb:aurora_data' already exists.
Upload complete.
Waiting on bqjob_r6888db82eb4c81ff_0000019ae330ec2f_1 ... (1s) Current status: DONE   


### Restart server

In [2]:
from google.cloud import bigquery
import pandas as pd
from google.cloud.bigquery import model
from google import genai

In [3]:
project_id = "qwiklabs-gcp-01-14276dd314fb"
location =  "us-central1"

#These are taken from above
mydataset = "aurora_data"
mytable = f"""{project_id}.{mydataset}.faqs_table"""
mymodel = f"""{project_id}.{mydataset}.sql_connected_model"""
embededTable = f"""{project_id}.{mydataset}.withEmbedding"""

In [23]:
## INIT
# Takes sql
client = bigquery.Client(project=project_id)

# Takes text
modelclient = genai.Client(vertexai=True, project=project_id, location=location)
chat = modelclient.chats.create(model="gemini-2.5-flash")


In [29]:
chat.send_message("hello")

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      avg_logprobs=-1.2799119026430192,
      content=Content(
        parts=[
          Part(
            text="""Hello there!

How can I help you today? Were you still wondering about that mayor, or is there something else you'd like to ask?"""
          ),
        ],
        role='model'
      ),
      finish_reason=<FinishReason.STOP: 'STOP'>
    ),
  ],
  create_time=datetime.datetime(2025, 12, 3, 7, 59, 36, 963286, tzinfo=TzInfo(UTC)),
  model_version='gemini-2.5-flash',
  response_id='6O0vadblOsnvhMIPodWP2AE',
  sdk_http_response=HttpResponse(
    headers=<dict len=9>
  ),
  usage_metadata=GenerateContentResponseUsageMetadata(
    candidates_token_count=31,
    candidates_tokens_details=[
      ModalityTokenCount(
        modality=<MediaModality.TEXT: 'TEXT'>,
        token_count=31
      ),
    ],
    prompt_token_count=28,
    prompt_tokens_details=[
      ModalityTokenCount(
      

In [5]:
## Makes a model that is connected to BigQuerry...
makeModel = f"""CREATE OR REPLACE MODEL `{mymodel}`
REMOTE WITH CONNECTION DEFAULT
OPTIONS (ENDPOINT = 'text-embedding-005');"""

client.query(makeModel)

QueryJob<project=qwiklabs-gcp-01-14276dd314fb, location=US, id=0b33fbec-5bb0-4f25-893d-0a5271b1cb18>

In [6]:
## Makes a Table that has been embeded
embedding = f"""
CREATE OR REPLACE TABLE `{embededTable}` AS
  SELECT *
  FROM ML.GENERATE_EMBEDDING(
  MODEL `{mymodel}`,
  (
    SELECT string_field_0, string_field_1,	CONCAT(string_field_0, string_field_1) AS content
    FROM `{mytable}`)
  );
"""
client.query(embedding)

QueryJob<project=qwiklabs-gcp-01-14276dd314fb, location=US, id=34d5b9bb-0a59-46d8-aa16-3024f0cbccc5>

In [12]:
%%bigquery df --project {project_id}
  SELECT *
  FROM aurora_data.faqs_table

Query is running:   0%|          |

Downloading:   0%|          |

In [13]:
df[:5]

,string_field_0,string_field_1
0,When was Aurora Bay founded?,Aurora Bay was founded in 1901 by a group of f...
1,What is the population of Aurora Bay?,Aurora Bay has a population of approximately 3...
2,Where is the Aurora Bay Town Hall located?,The Town Hall is located at 100 Harbor View Ro...
3,Who is the current mayor of Aurora Bay?,"The current mayor is Linda Greenwood, elected ..."
4,What are the primary industries in Aurora Bay?,The primary industries include commercial fish...


In [10]:
# This is just a hard coded print check
%%bigquery df --project {project_id}
  SELECT *
  FROM aurora_data.withEmbedding

Query is running:   0%|          |

Downloading:   0%|          |

In [11]:
df[:5]


,ml_generate_embedding_result,ml_generate_embedding_statistics,ml_generate_embedding_status,string_field_0,string_field_1,content
0,"[-0.02901100181043148, 0.013011284172534943, 0...","{""token_count"":29,""truncated"":false}",,What are the primary industries in Aurora Bay?,The primary industries include commercial fish...,What are the primary industries in Aurora Bay?...
1,"[-0.02901100181043148, 0.013011284172534943, 0...","{""token_count"":29,""truncated"":false}",,What are the primary industries in Aurora Bay?,The primary industries include commercial fish...,What are the primary industries in Aurora Bay?...
2,"[-0.02901100181043148, 0.013011284172534943, 0...","{""token_count"":29,""truncated"":false}",,What are the primary industries in Aurora Bay?,The primary industries include commercial fish...,What are the primary industries in Aurora Bay?...
3,"[-0.08013758808374405, -0.03401567041873932, -...","{""token_count"":29,""truncated"":false}",,Who is the current mayor of Aurora Bay?,"The current mayor is Linda Greenwood, elected ...",Who is the current mayor of Aurora Bay?The cur...
4,"[-0.08013758808374405, -0.03401567041873932, -...","{""token_count"":29,""truncated"":false}",,Who is the current mayor of Aurora Bay?,"The current mayor is Linda Greenwood, elected ...",Who is the current mayor of Aurora Bay?The cur...


Bot can use VECTOR_SEARCH to find an embedding that matches the input

```
SELECT base.string_field_1, base.string_field_0
FROM VECTOR_SEARCH(
  TABLE `qwiklabs-gcp-01-14276dd314fb.aurora_data.withEmbedding`,
  'ml_generate_embedding_result',
  (
    SELECT ml_generate_embedding_result, content AS query
    FROM ML.GENERATE_EMBEDDING(
      MODEL `qwiklabs-gcp-01-14276dd314fb.aurora_data.mymodel`,
      (SELECT 'Who is the mayor?' AS content)
      )
  ),
  top_k => 5,
  options => '{"fraction_lists_to_search": 0.01}'
)
```
* base. is the og before embeding.
* base.query is used instead of query.query because q.q is just user input.

### Subquery
Using mymodel to embed the user prompt and outputed as var content
Take content (return as query for above) and it's new embeding col
```
    SELECT ml_generate_embedding_result, content AS query
    FROM ML.GENERATE_EMBEDDING(
      MODEL `qwiklabs-gcp-01-14276dd314fb.aurora_data.mymodel`,
      (SELECT 'Who is the mayor?' AS content)
      )
```

In [48]:
def rag_query(user_input):
  answer = "string_field_1"
  question = "string_field_0"

  ## Embed user query and find closest neighbors
  instructions = (f"""
    SELECT base.{answer},	base.{question}
    FROM VECTOR_SEARCH(
      TABLE `{embededTable}`,
      'ml_generate_embedding_result',
      (
        SELECT ml_generate_embedding_result, content AS query
        FROM ML.GENERATE_EMBEDDING(
          MODEL `{mymodel}`,
            (SELECT '{user_input}' AS content))
      ),
    top_k => 5,""" +
    """options => '{"fraction_lists_to_search": 0.01}'
    )
    """
  )
  #print(instructions)

  ## Query BQ
  res = client.query(instructions)
  if res == None:
    print(":( sorry no luck")

  ## Build context
  context = ""
  for row in res:
    context += row['string_field_1'] + "\n"

  final_prompt = f"""
    You work at the front desk of Aurora and answer common faq quesiont about the town.
    Given the context {context}
    briefly answer the users question: {user_input}
    """

  response = chat.send_message(final_prompt)
  return response.text

In [49]:

user_input = "Who is the mayor?"
rag_query(user_input)


'The current mayor is Linda Greenwood. She was elected in 2021 for a four-year term.'

In [50]:

user_input = "places to hike?"
rag_query(user_input)


'You can hike in the nearby forests. North Star Excursions and Bay Explorers also offer guided hikes.'